In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import yaml
import matplotlib.pyplot as plt
from pycm import *
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

from keras.layers import Dense, Flatten, AveragePooling2D, Dropout, MaxPool2D
from tensorflow.keras.optimizers import Adam
from keras.applications.xception import Xception
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.models import Model
from keras.layers import Dense, MaxPool2D, Conv2D


In [ ]:
!pip install -U pyYAML
!pip install pycm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 5.4 MB/s 
  Attempting uninstall: pyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 66 kB 2.9 MB/s 
     |████████████████████████████████| 590 kB 11.2 MB/s 


In [ ]:
cfg = yaml.full_load(open("/content/drive/MyDrive/FP/config.yml", 'r'))

In [ ]:
train_data = pd.read_csv(cfg['PATHS']['TRAIN_SET'])
test_data = pd.read_csv(cfg['PATHS']['TEST_SET'])
valid_data = pd.read_csv(cfg['PATHS']['VALID_SET'])

In [ ]:
train_data_gen = ImageDataGenerator(
    preprocessing_function= preprocess_input, 
    zoom_range= 0.2, 
    horizontal_flip= True, 
    shear_range= 0.2,      
)


In [ ]:
train_generator = train_data_gen.flow_from_dataframe(
    train_data,
    x_col='filename',
    y_col='category_str',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=128
)

Found 2199 validated image filenames belonging to 3 classes.


In [ ]:
test_data_gen = ImageDataGenerator(preprocessing_function= preprocess_input  )

test_generator = test_data_gen.flow_from_dataframe(
    test_data,
    x_col='filename',
    y_col='category_str',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=128,
    shuffle = False,
    seed=42

)

Found 733 validated image filenames belonging to 3 classes.


In [ ]:
valid_data_gen = ImageDataGenerator(preprocessing_function= preprocess_input )

valid_generator = valid_data_gen.flow_from_dataframe(
    valid_data,
    x_col='filename',
    y_col='category_str',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=128
)

Found 733 validated image filenames belonging to 3 classes.


In [ ]:
res = ResNet50( input_shape=(224,224,3), include_top= False)

for layer in res.layers:           # Dont Train the parameters again 
  layer.trainable = False

x = Flatten()(res.output)
x = Dense(units=3 , activation='sigmoid', name = 'predictions' )(x)

# creating our model.
model = Model(res.input, x)

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
model.compile( optimizer= 'adam' , loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

es = EarlyStopping(monitor= "val_accuracy" , min_delta= 0.01, patience= 3, verbose=1)
mc = ModelCheckpoint(filepath="bestmodel.h5", monitor="val_accuracy", verbose=1, save_best_only= True)

In [ ]:
hist = model.fit_generator(train_generator , steps_per_epoch= 10, epochs= 30, validation_data= valid_generator , validation_steps= 16, callbacks=[es,mc])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/30
10/10 [==============================] - ETA: 0s - loss: 3.7333 - accuracy: 0.8349 WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 16 batches). You may need to use the repeat() function when building your dataset.

Epoch 1: val_accuracy improved from -inf to 0.93452, saving model to bestmodel.h5
10/10 [==============================] - 411s 45s/step - loss: 3.7333 - accuracy: 0.8349 - val_loss: 1.4691 - val_accuracy: 0.9345
Epoch 2/30
10/10 [==============================] - 107s 11s/step - loss: 2.2871 - accuracy: 0.8843
Epoch 3/30
10/10 [==============================] - 61s 6s/step - loss: 1.8091 - accuracy: 0.8996
Epoch 4/30
10/10 [==============================] - 37s 4s/step - loss: 1.1145 - accuracy: 0.9234
Epoch 5/30
10/10 [==============================] - 26s 3s/step - loss: 0.9242 - accuracy: 0.9398
Epoch 6/30
10/10 [==============

In [ ]:
model.save(cfg['PATHS']['RESNET'])

### Statistics of the Model

In [ ]:
from keras.models import load_model
model = load_model("/content/drive/MyDrive/FP/models/resnet_model.h5")

In [ ]:
acc = model.evaluate_generator(generator= test_generator)[1]
print(f"The accuracy of your model is = {acc} %")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


The accuracy of your model is = 0.9645293354988098 %


In [ ]:
y_pred = model.predict(test_generator,verbose =0)


In [ ]:
y_pred = np.argmax(y_pred,axis =-1)
y_pred

y_pred.shape

(733,)

In [ ]:
test_generator.class_indices

{'Covid': 0, 'Normal': 1, 'Viral Pneumonia': 2}

In [ ]:
test_labels = test_generator.labels
cm = ConfusionMatrix(actual_vector=test_labels, predict_vector=y_pred)

In [ ]:
cm.print_matrix()

Predict   0         1         2         
Actual
0         214       1         3         

1         12        227       7         

2         0         3         266       




In [ ]:
test_generator.class_indices

{'Covid': 0, 'Normal': 1, 'Viral Pneumonia': 2}

In [ ]:
cm.stat(summary=True)

Overall Statistics : 

ACC Macro                                                         0.97635
F1 Macro                                                          0.96396
FPR Macro                                                         0.01769
Kappa                                                             0.94661
Overall ACC                                                       0.96453
PPV Macro                                                         0.96445
SOA1(Landis & Koch)                                               Almost Perfect
TPR Macro                                                         0.96442
Zero-one Loss                                                     26

Class Statistics :

Classes                                                           0             1             2             
ACC(Accuracy)                                                     0.97817       0.96862       0.98226       
AUC(Area under the ROC curve)                                     0.97

In [ ]:
print(classification_report(test_labels,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.98      0.96       218
           1       0.98      0.92      0.95       246
           2       0.96      0.99      0.98       269

    accuracy                           0.96       733
   macro avg       0.96      0.96      0.96       733
weighted avg       0.97      0.96      0.96       733

